In [4]:
import pandas as pd
import pydeck as pdk
import glob
import geopandas as gpd
from shapely.geometry import Polygon, Point
from shapely.wkt import loads

"""
# Shape 파일을 GeoDataFrame으로 불러와서 저장.
file_path = glob.glob("./data/*.zip")[0]
gdf = gpd.GeoDataFrame.from_file(file_path, encoding='CP949')
gdf = gdf.to_crs({"init": "epsg:4326"})
new_file_name = "서울_건물_데이터.csv"
gdf.to_csv(f"./data/{new_file_name}", encoding='utf8', index=False)
"""

"""
# 서울시 건물 데이터 file을 가져온다.
file_path = '서울_건물_데이터.csv'
seoulbuilding_data = pd.read_csv(file_path)

# 건물 데이터를 연면적과 층수, 도형좌표계로 정리한다. 결측치는 0으로 정리.
seoulbuilding_df = pd.DataFrame({'연면적': seoulbuilding_data['TOTALAREA'], '지상층수': seoulbuilding_data['GRND_FLR'], '도형좌표': seoulbuilding_data['geometry']})
seoulbuilding_df = seoulbuilding_df.fillna(0)

# 빌딩 등록현황을 저장할 데이터프레임 선언.
building_df = pd.DataFrame(columns = ['연면적', '지상층수', '경도', '위도'])


# 도형의 좌표중심을 구하고 대형건물만 뽑아서 저장한다.
# 대형 건물은 연면적 10만m^2 이상 혹은 21층 이상의 건물이 해당된다.
for i in range(0, len(seoulbuilding_df)):
    totalarea = seoulbuilding_df.iloc[i]['연면적']
    ground_ftr = seoulbuilding_df.iloc[i]['지상층수']
    # 데이터가 날짜로 오기된 것을 배제해야함
    if ((totalarea >= 100000) and (totalarea < 20000000)) or (ground_ftr >= 21):
        polygon_str = seoulbuilding_df.iloc[i]['도형좌표'].replace('POLYGON ((', '').replace('))', '').split(', ')
        coordinates = []
        for coord in polygon_str:
            if coord[0] == '(':
                coord = coord[1:]
            elif coord[-1] == ')':
                coord = coord[:-1]
            coordinates.append(tuple(map(float, coord.split())))

        polygon = Polygon(coordinates)
        center = polygon.centroid

        new_row = pd.DataFrame({'연면적': [totalarea], '지상층수': [ground_ftr], '경도': [center.x], '위도': [center.y]})
        building_df = pd.concat([building_df, new_row])

        print(i)

building_df.to_csv('서울시_대형건물_좌표.csv', encoding='cp949')
"""


# 빌딩 등록현황을 저장할 데이터프레임 선언.
seoul_large_building_data = pd.read_csv('서울시_대형건물_좌표.csv', encoding='cp949')
building_df = pd.DataFrame(seoul_large_building_data)
building_df = building_df.drop(columns=['Unnamed: 0'])



print(building_df[:3])

view_state = pdk.ViewState(latitude=37.6, longitude=126.9, zoom=11, bearing=0, pitch=45)

# Define a layer to display on a map
layer = pdk.Layer(
    'ScatterplotLayer',
    building_df,
    get_position=['경도', '위도'],
    get_radius=100,
    get_fill_color=[255, 0, 0],
    pickable=True,
    #threshold=1,
    #extruded = True,
    #elevation_scale = 1,
    #cell_size = 200,
)

# Render
r = pdk.Deck(
    layers=[layer],
    initial_view_state=view_state,
    # views=[pdk.View(type="MapView", controller=True)],  # 지도 상호 작용을 위해 views 속성 추가
)
r.to_html()

       연면적  지상층수          경도         위도
0  96516.0  32.0  126.876757  37.532187
1  10865.0  33.0  127.091427  37.513512
2  31446.0  23.0  127.014471  37.574454
